# Exploring the wavelets

A quick debugging notebook to look at the waveletes given some specific set of rock/pipe params.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
from matplotlib import pyplot as plt
import matplotlib
from matplotlib.cm import get_cmap

cmap = get_cmap('bwr')

plt.style.use('seaborn-dark')

plt.rcParams['axes.grid'] = True
plt.rcParams['figure.figsize'] = (15, 5)

from scipy import signal
from itertools import product

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm

In [ ]:
from theory.core import TheoreticalWavelet, Pipe, Rock

In [ ]:
pipe = Pipe(Rb=.16, alpha=4875, rho=7200)
rock = Rock(2000, 1500)
wavelet = TheoreticalWavelet(pipe, rock, filter_duration=.1)

primary = wavelet.primary_in_time_domain(200, None, filtered=True)
reflected = wavelet.reflected_in_time_domain(200, None, filtered=True)
multiple = wavelet.multiple_in_time_domain(200, None, filtered=True)

time = wavelet.get_time_range_for_window(200)

fig, axes = plt.subplots(1, 3, figsize=(15,7.5/2.5), sharey=False)
axes[0].plot(time, primary, label='primary', color='green')
axes[0].set_title('Primary in Time Domain')
axes[0].legend()
axes[1].plot(time, reflected)
axes[1].set_title('Reflected in Time Domain')
axes[2].plot(time, multiple, color='salmon')
axes[2].set_title('Multiple in Time Domain')
for ax in axes.ravel():
    ax.set_xlabel('Time (ms)')
fig.suptitle('Rock_alpha: {} - Rock_rho: {} - Pipe_Rb: {}'.format(rock.alpha, rock.rho, pipe.Rb), y=1.05)
fig.tight_layout()

In [ ]:
wavelet.sampling_rate

In [ ]:
wavelet.sampling_interval*1000

In [ ]:
p = wavelet.primary_in_time_domain(None, None, filtered=True)

In [ ]:
pd.Series(p).describe()

In [ ]:
pacorr = signal.correlate(p, p, method='fft')
h = int(len(pacorr)/2)
W = pacorr[h:h+4000]
W[0] = W[0]+1

In [ ]:
from scipy.linalg import toeplitz, inv

In [ ]:
A = toeplitz(pacorr)

In [ ]:
b = np.r_[1, [0] * 3999]

In [ ]:
c = inv(A)

In [ ]:
W.shape

In [ ]:
c.shape

In [ ]:
W

In [ ]:
np.dot(W/1e6, c)

In [ ]:
W[:, None] c

In [ ]:
b.shape

In [ ]:
ax = plt.axes()
ax.matshow(np.log(c))

In [ ]:
ax = plt.axes()
ax.plot(pacorr[h:h+4000])
ax.figure.set_size_inches(35, 10)